# Using Mixtures

The notebook explains how to use the Mixture classes to create *Phase* objects with the thermophysical properties arising from the properties and compositions of the individual components.

In [1]:
import openpnm as op
pn = op.network.Demo()

## Create individual species objects

Let's illustrate this
by creating ``air`` as a mixture of oxygen and nitrogen.  First we create a ``Phase`` object for each individual species.  OpenPNM uses a package called [chemicals](https://pypi.org/project/chemicals/) to provide physical properties for a large variety of pure species. So we can create a ``Species`` object "by name" which will fetch the corresponding properties from a database:

In [3]:
o2 = op.phase.StandardGas(network=pn, species='oxygen')
n2 = op.phase.StandardGas(network=pn, species='nitrogen')

The ``species`` argument accepts a variety of names for a given species, so that ``species='o2'`` would also work.  We can print these objects to see what properties were calculated:

In [4]:
print(o2)


══════════════════════════════════════════════════════════════════════════════
phase_01 : <openpnm.phase.StandardGas at 0x2b6a15bd270>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.density                                                        9 / 9
  2  pore.heat_capacity                                                  9 / 9
  3  pore.heat_capacity_gas                                              9 / 9
  4  pore.pressure                                                       9 / 9
  5  pore.temperature                                                    9 / 9
  6  pore.thermal_conductivity                                           9 / 9
  7  pore.viscosity                                                      9 / 9
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

This computes a few basic properties, but more importantly, the ``params`` attribute is loaded with a physical constants:

In [5]:
print(o2.params)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Parameters                          Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
CAS                                 7782-44-7
common_name                         oxygen
charge                              0
formula                             O2
boiling_temperature                 90.188
melting_temperature                 54.36
triple_point_temperature            54.33
triple_point_pressure               148.9796864589355
dipole_moment                       0.0
LJ_diameter                         3.29728
LJ_energy                           1.6520845934e-21
surface_tension_Tb                  0.013145633010272155
molar_volume_Tb                     2.802254619756072e-05
molecular_weight                    31.9988
critical_temperature                154.58
critical_pressure                   5042945.25
critical_volume                     7.34e-05
critical_compressibilt

These constants are used for computation of more complex properties, especially for mixtures.

----
💡 **Pro Tip**: You can access the values of these parameters either from the ``params`` attribute or using the dictionary lookup of the main object which will dispatch the query to the ``params`` attribute. 

In [6]:
o2.params['CAS']

'7782-44-7'

In [7]:
o2['param.CAS']

'7782-44-7'

Writing also works.

In [ ]:
o2['param.foo'] = 'bar'
o2.params['foo']

---

## Create a mixture object

The ``phase`` module contains a few classes designed to work with mixtures.  These basically keep track of which individual species are in the mixture and their compositions, and they have a suite of pore-scale models that compute the mixture properties using common mixing rules or correlations:

In [9]:
air = op.phase.StandardGasMixture(network=pn, components=[o2, n2])

The ``air`` object is not quite ready yet because we have not specified the composition of each species.  This can be done a few  ways, but a useful and powerful method is available, ``y``, which allows both *setting* the mole fraction (conventionally referred to as *y* in chemical engineering) and *getting* the current values. When a component name *and* a mole fraction is given it sets this value:

In [10]:
air.y(o2.name, 0.21)
air.y(n2.name, 0.79)

The values of mole fraction are stored in the ``Mixture`` object with the species names as part of the key:

In [11]:
print(air['pore.mole_fraction.'+o2.name])
print(air['pore.mole_fraction.'+n2.name])

[0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21]
[0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79]


----
💡 **ProTip!** The mole fractions of *both* species can be retrieved using the `dict` lookup without specifying which component.  OpenPNM will return a dictionary of both components with their names as they keys:

In [12]:
print(air['pore.mole_fraction'])

{'phase_01': array([0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21]), 'phase_02': array([0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79])}


💡 This also means you can index into the returned dictionary using the names:

In [13]:
print(air['pore.mole_fraction'][o2.name])

[0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21]


💡 Alternatively the returned dictionary can be used to get a list of components that are currently part of the mixture:

In [15]:
air['pore.mole_fraction'].keys()

dict_keys(['phase_01', 'phase_02'])

💡 The individual species can be retrieved using the ``components`` attribute which returns a dictionary with component names as the keys and handles to the actual objects as values:

In [16]:
air.components

{'phase_01': phase_01 : <openpnm.phase.StandardGas at 0x2b6a15bd270>,
 'phase_02': phase_02 : <openpnm.phase.StandardGas at 0x2b6a71eb400>}

----

The ``y`` method (and ``x`` on ``LiquidMixture``) can also be used to retrieve the mole fraction of one or both components:

In [17]:
air.y()

{'phase_01': array([0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21, 0.21]),
 'phase_02': array([0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79])}

In [18]:
air.y(n2.name)

array([0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79])

In addition to keeping track of the species and their compositions, the mixture object also computes mixture properties. These can be seen by printing ``air``.  Note that we must first called ``regenerate_models`` since the models were not run on instantiation since the mole fraction was not set:

In [22]:
air.regenerate_models()
print(air)


══════════════════════════════════════════════════════════════════════════════
mixture_01 : <openpnm.phase.StandardGasMixture at 0x2b6a2682bd0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.density                                                        9 / 9
  2  pore.diffusivity                                                    9 / 9
  3  pore.heat_capacity                                                  9 / 9
  4  pore.mole_fraction.phase_01                                         9 / 9
  5  pore.mole_fraction.phase_02                                         9 / 9
  6  pore.pressure                                                       9 / 9
  7  pore.temperature                                                    9 / 9
  8  pore.thermal_conductivity                                   

Several of the properties listed above were on the species objects, such as viscosity.  These viscosity of the mixture is computed using a mole-weighted approach. The molecular weight is also the mole-weighted average of the two species.  Diffusivity on the other hand was not present on the species.  It is computed for the mixture using correlations. The binary diffusivity computing for air (i.e oxygen in nitrogen) is:

In [23]:
air['pore.diffusivity']

array([2.13367825e-05, 2.13367825e-05, 2.13367825e-05, 2.13367825e-05,
       2.13367825e-05, 2.13367825e-05, 2.13367825e-05, 2.13367825e-05,
       2.13367825e-05])

which is only slightly higher than the commonly accepted value of 2.09e-5.

## Other features of mixtures

The various mixture classes offer a few other features, which will be explored below.

### Adding and Removing Species

You can remove a species:

In [24]:
air.remove_comp(o2.name)
air.components

{'phase_02': phase_02 : <openpnm.phase.StandardGas at 0x2b6a71eb400>}

----
💡 **ProTip!** A species is considered a component of a mixture *if and only if* ``'pore.mole_fraction.<species.name>'`` appears in the mixture dictionary. Adding and removing the corresponding array from the dictionary is literally how the components are defined. For instance:

In [25]:
del air['pore.mole_fraction.' + n2.name]
air.components

{}

They can be readded in the same way:

In [26]:
air['pore.mole_fraction.' + n2.name] = 0.79
air.components

{'phase_02': phase_02 : <openpnm.phase.StandardGas at 0x2b6a71eb400>}

But there is a specific method for this:

In [27]:
air.add_comp(o2, mole_fraction=0.21)
air.components

{'phase_02': phase_02 : <openpnm.phase.StandardGas at 0x2b6a71eb400>,
 'phase_01': phase_01 : <openpnm.phase.StandardGas at 0x2b6a15bd270>}

----

### Check Consistency of Mixture

You can also check the health of the mixture, such as whether all the mole fractions add to 1.0 each each pore:

In [ ]:
print(air.check_mixture_health())

In [29]:
air.y(o2.name, 0.1)
print(air.check_mixture_health())

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Key                                 Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
mole_fraction_too_low               (9,)
mole_fraction_too_high              []
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


### Retrieving Species Properties

Lastly, the properties of the individual species can be retrieved from the mixture as follows:

In [ ]:
air.get_comp_vals('pore.viscosity')

----
💡 **ProTip!**  It is also possible to use the ``*`` notation as follows:

In [28]:
air['pore.viscosity.*']

{'phase_02': array([1.69779528e-05, 1.69779528e-05, 1.69779528e-05, 1.69779528e-05,
        1.69779528e-05, 1.69779528e-05, 1.69779528e-05, 1.69779528e-05,
        1.69779528e-05]),
 'phase_01': array([2.09391006e-05, 2.09391006e-05, 2.09391006e-05, 2.09391006e-05,
        2.09391006e-05, 2.09391006e-05, 2.09391006e-05, 2.09391006e-05,
        2.09391006e-05])}

The reason the ``*`` notation is needed is that ``'pore.viscosity'`` is already taken by the mixture viscosity.  Ideally, ``air['pore.viscosity']`` should trigger the lookup of the species values. It was briefly considered that all mixture properties should be labelled ``'pore.viscosity.mixture'``, which would have allowed for ``air['pore.viscosity']`` to trigger the return of the mixture viscosity and all components in a dictionary, but this added a lot of complications behind the scenes so the idea was dropped. As a work around, the ``*`` notation was added to the ``__getitem__`` method such that any dictionary lookups with a key that ends in ``*`` trigger a lookup of the requested property on any component species.

----